In [36]:
import pandas as pd 
import yfinance as yf
from IPython.display import display, Markdown

In [37]:
ticker_symbol = "AAPL"
expiration_index = 0

# Create and load the ticker
ticker = yf.Ticker(ticker_symbol)
expirations = ticker.options
selected_expiration = expirations[expiration_index]

print(f"Using expiration date: {selected_expiration}")

# Get option chain
option_chain = ticker.option_chain(selected_expiration)
calls = option_chain.calls
puts = option_chain.puts

# Normalize IV
def normalize_iv(df):
    df = df.copy()
    df['impliedVolatility'] = df['impliedVolatility'].astype(float)
    df['impliedVolatility'] = df['impliedVolatility'].apply(
        lambda x: x / 100 if x > 2 else x
    )
    return df

calls = normalize_iv(calls)
puts = normalize_iv(puts)

# Filter stirkes to get rid of outliers
current_price = ticker.info['regularMarketPrice']

calls = calls[(calls['strike'] >= current_price * 0.8) &
              (calls['strike'] <= current_price * 1.2)]
puts = puts[(puts['strike'] >= current_price * 0.8) &
            (puts['strike'] <= current_price * 1.2)]

# Format cleaned data
display(Markdown("Calls (Filtered):"))
display(calls[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility']].head())

display(Markdown("Puts (Filtered):"))
display(puts[['contractSymbol', 'strike', 'lastPrice', 'bid', 'ask', 'impliedVolatility']].head())


Using expiration date: 2025-08-01


Calls (Filtered):

,contractSymbol,strike,lastPrice,bid,ask,impliedVolatility
10,AAPL250801C00170000,170.0,40.17,38.10,40.80,1.628908
11,AAPL250801C00175000,175.0,35.00,32.25,36.05,1.136723
12,AAPL250801C00180000,180.0,32.30,28.40,30.85,1.351566
13,AAPL250801C00182500,182.5,28.20,25.75,28.65,1.290043
14,AAPL250801C00185000,185.0,24.85,23.30,25.90,1.141606


Puts (Filtered):

,contractSymbol,strike,lastPrice,bid,ask,impliedVolatility
11,AAPL250801P00170000,170.0,0.01,0.01,0.03,1.062505
12,AAPL250801P00175000,175.0,0.05,0.03,0.04,0.992188
13,AAPL250801P00180000,180.0,0.07,0.06,0.07,0.917970
14,AAPL250801P00182500,182.5,0.09,0.07,0.09,0.871095
15,AAPL250801P00185000,185.0,0.11,0.10,0.12,0.835939
